<a href="https://colab.research.google.com/github/ajiepamungkasep/Twitter-Sentiment-Analysis/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.3 MB/s 


Import module

In [ ]:
import pandas as pd
import re
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Import csv

In [ ]:
tweet_df = pd.read_csv('polisi.csv', encoding='utf-8')
tweet_df

,Unnamed: 0,text
0,0,@fadlizon Pak bukannya DPR juga menganggarkan ...
1,1,@txtdrberseragam KNP JADI POLISI ANJG 😭
2,2,Jumlah anggota Polri yang diperiksa terkait Tr...
3,3,Gduwe doyo maneh gae menghakimi polisi yaallah...
4,4,Padahal orang tua nya di bunuh polisi https://...
...,...,...
988,988,Polisi sibuk bikin video humanis
989,989,"Najwa Shihab akan Laporkan Kepada Polisi, Niki..."
990,990,"Tiba di Malang, Kapolri Jenguk Korban Tragedi ..."
991,991,Retweet &amp; Follow Biar Ga Hilang\n--\nKolek...


In [ ]:
df_slang = pd.read_csv('slang.csv',sep=';')
df_slang

,slang,formal
0,demen,suka
1,mlh,malah
2,gn,gunung
3,bgni,begini
4,naon,apa
...,...,...
2100,woles,santai
2101,wtf,apa-apaan
2102,x,kali
2103,y,ya


Data cleansing

In [ ]:
def cleansing(tweet):
    if type(tweet) == np.float:
        return ""
    tweet = tweet.lower() #change teks into lower case
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet) #remove mentions
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet) #remove hashtag
    tweet = re.sub(r'http\S+', '', tweet) #remove link
    tweet = re.sub(r'[?|$|.|@#%^/&*=!_:")(-+,]','',tweet) #remove symbol
    tweet = re.sub(r'[0-9]+','',tweet) #remove number
    tweet = tweet.replace('\n',' ') #change new line to space
    tweet = tweet.strip(' ') #remove space between the text
    return tweet

tweet_df['text_clean'] = tweet_df['text'].apply(cleansing)
tweet_df.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,Unnamed: 0,text,text_clean
0,0,@fadlizon Pak bukannya DPR juga menganggarkan ...,pak bukannya dpr juga menganggarkan gas air ma...
1,1,@txtdrberseragam KNP JADI POLISI ANJG 😭,knp jadi polisi anjg 😭
2,2,Jumlah anggota Polri yang diperiksa terkait Tr...,jumlah anggota polri yang diperiksa terkait tr...
3,3,Gduwe doyo maneh gae menghakimi polisi yaallah...,gduwe doyo maneh gae menghakimi polisi yaallah...
4,4,Padahal orang tua nya di bunuh polisi https://...,padahal orang tua nya di bunuh polisi


In [ ]:
def replace_slang(text_clean):
  res = ''
  for item in text_clean.split():
    if item in df_slang.slang.values:
      res += df_slang[df_slang['slang'] == item]['formal'].iloc[0]
    else:
      res += item
    res += ' '
  return res

tweet_df['text_clean'] = tweet_df['text_clean'].apply(replace_slang)
tweet_df

,Unnamed: 0,text,text_clean
0,0,@fadlizon Pak bukannya DPR juga menganggarkan ...,pak bukannya dpr juga menganggarkan gas air ma...
1,1,@txtdrberseragam KNP JADI POLISI ANJG 😭,kenapa jadi polisi anjing 😭
2,2,Jumlah anggota Polri yang diperiksa terkait Tr...,jumlah anggota polri yang diperiksa terkait tr...
3,3,Gduwe doyo maneh gae menghakimi polisi yaallah...,gduwe doyo kamu gae menghakimi polisi yaallah ...
4,4,Padahal orang tua nya di bunuh polisi https://...,padahal orang tua nya di bunuh polisi
...,...,...,...
988,988,Polisi sibuk bikin video humanis,polisi sibuk membuat video humanis
989,989,"Najwa Shihab akan Laporkan Kepada Polisi, Niki...",najwa shihab akan laporkan kepada polisi nikit...
990,990,"Tiba di Malang, Kapolri Jenguk Korban Tragedi ...",tiba di malang kapolri jenguk korban tragedi k...
991,991,Retweet &amp; Follow Biar Ga Hilang\n--\nKolek...,retweet amp; follow agar tidak hilang -- kolek...


In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

stop_factory = StopWordRemoverFactory().get_stop_words()
stp = stop_factory
dictionary = ArrayDictionary(stp)
stopword = StopWordRemover(dictionary)

tweet_df['text_clean'] = tweet_df.text_clean.apply(lambda x : stopword.remove(x))
tweet_df

,Unnamed: 0,text,text_clean
0,0,@fadlizon Pak bukannya DPR juga menganggarkan ...,pak bukannya dpr menganggarkan gas air mata te...
1,1,@txtdrberseragam KNP JADI POLISI ANJG 😭,jadi polisi anjing 😭
2,2,Jumlah anggota Polri yang diperiksa terkait Tr...,jumlah anggota polri diperiksa terkait tragedi...
3,3,Gduwe doyo maneh gae menghakimi polisi yaallah...,gduwe doyo kamu gae menghakimi polisi yaallah ...
4,4,Padahal orang tua nya di bunuh polisi https://...,padahal orang tua nya bunuh polisi
...,...,...,...
988,988,Polisi sibuk bikin video humanis,polisi sibuk membuat video humanis
989,989,"Najwa Shihab akan Laporkan Kepada Polisi, Niki...",najwa shihab laporkan polisi nikita mirzani ej...
990,990,"Tiba di Malang, Kapolri Jenguk Korban Tragedi ...",tiba malang kapolri jenguk korban tragedi kanj...
991,991,Retweet &amp; Follow Biar Ga Hilang\n--\nKolek...,retweet amp; follow tidak hilang -- koleksi ci...


In [ ]:
def tokenizing(text):
  text = word_tokenize(text)
  return text

tweet_df['text_preprocessed'] = tweet_df['text_clean'].apply(tokenizing)
tweet_df

,Unnamed: 0,text,text_clean,text_preprocessed
0,0,@fadlizon Pak bukannya DPR juga menganggarkan ...,pak bukannya dpr menganggarkan gas air mata te...,"[pak, bukannya, dpr, menganggarkan, gas, air, ..."
1,1,@txtdrberseragam KNP JADI POLISI ANJG 😭,jadi polisi anjing 😭,"[jadi, polisi, anjing, 😭]"
2,2,Jumlah anggota Polri yang diperiksa terkait Tr...,jumlah anggota polri diperiksa terkait tragedi...,"[jumlah, anggota, polri, diperiksa, terkait, t..."
3,3,Gduwe doyo maneh gae menghakimi polisi yaallah...,gduwe doyo kamu gae menghakimi polisi yaallah ...,"[gduwe, doyo, kamu, gae, menghakimi, polisi, y..."
4,4,Padahal orang tua nya di bunuh polisi https://...,padahal orang tua nya bunuh polisi,"[padahal, orang, tua, nya, bunuh, polisi]"
...,...,...,...,...
988,988,Polisi sibuk bikin video humanis,polisi sibuk membuat video humanis,"[polisi, sibuk, membuat, video, humanis]"
989,989,"Najwa Shihab akan Laporkan Kepada Polisi, Niki...",najwa shihab laporkan polisi nikita mirzani ej...,"[najwa, shihab, laporkan, polisi, nikita, mirz..."
990,990,"Tiba di Malang, Kapolri Jenguk Korban Tragedi ...",tiba malang kapolri jenguk korban tragedi kanj...,"[tiba, malang, kapolri, jenguk, korban, traged..."
991,991,Retweet &amp; Follow Biar Ga Hilang\n--\nKolek...,retweet amp; follow tidak hilang -- koleksi ci...,"[retweet, amp, ;, follow, tidak, hilang, --, k..."


In [ ]:
def stemming(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  text = [stemmer.stem(word) for word in text]
  return text

tweet_df['text_preprocessed'] = tweet_df['text_preprocessed'].apply(stemming)
tweet_df

,Unnamed: 0,text,text_clean,text_preprocessed
0,0,@fadlizon Pak bukannya DPR juga menganggarkan ...,pak bukannya dpr menganggarkan gas air mata te...,"[pak, bukan, dpr, anggar, gas, air, mata, sebu..."
1,1,@txtdrberseragam KNP JADI POLISI ANJG 😭,jadi polisi anjing 😭,"[jadi, polisi, anjing, ]"
2,2,Jumlah anggota Polri yang diperiksa terkait Tr...,jumlah anggota polri diperiksa terkait tragedi...,"[jumlah, anggota, polri, periksa, kait, traged..."
3,3,Gduwe doyo maneh gae menghakimi polisi yaallah...,gduwe doyo kamu gae menghakimi polisi yaallah ...,"[gduwe, doyo, kamu, gae, hakim, polisi, yaalla..."
4,4,Padahal orang tua nya di bunuh polisi https://...,padahal orang tua nya bunuh polisi,"[padahal, orang, tua, nya, bunuh, polisi]"
...,...,...,...,...
988,988,Polisi sibuk bikin video humanis,polisi sibuk membuat video humanis,"[polisi, sibuk, buat, video, humanis]"
989,989,"Najwa Shihab akan Laporkan Kepada Polisi, Niki...",najwa shihab laporkan polisi nikita mirzani ej...,"[najwa, shihab, lapor, polisi, nikita, mirzani..."
990,990,"Tiba di Malang, Kapolri Jenguk Korban Tragedi ...",tiba malang kapolri jenguk korban tragedi kanj...,"[tiba, malang, kapolri, jenguk, korban, traged..."
991,991,Retweet &amp; Follow Biar Ga Hilang\n--\nKolek...,retweet amp; follow tidak hilang -- koleksi ci...,"[retweet, amp, , follow, tidak, hilang, --, ko..."


In [ ]:
tweet_df.to_csv('polisi_preprocessed.csv')